In [4]:
import numpy as np
filename = "Gemini.txt" 
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i)  for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total chars :", (n_chars))
print("Total vocab :", (n_vocab))

Total chars : 5429
Total vocab : 45


In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)